In [86]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
# from pandas_profiling import ProfileReport
import numpy as np
import warnings
import mlflow
from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

ROOT = os.environ.get('PWD')

warnings.filterwarnings('ignore')






# Data cleaning and feature engineering

In [87]:
# read in new tiny master

new_tiny_master = pd.read_csv('../data//new_tiny_master.csv')

In [88]:
# extract the data for arrival date in 2019 only

tiny_master_20 = new_tiny_master.query("year == 2020")

In [89]:
tiny_master_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 699077 entries, 12 to 1117200
Data columns (total 86 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      699077 non-null  int64  
 1   listing_id                      699077 non-null  object 
 2   yearmonth                       699077 non-null  object 
 3   result_views                    699077 non-null  float64
 4   expose_views                    699077 non-null  float64
 5   year                            699077 non-null  int64  
 6   month                           699077 non-null  int64  
 7   1_room_apartment                699077 non-null  float64
 8   dining_room                     699077 non-null  float64
 9   en_suite_bathroom               699077 non-null  float64
 10  kitchen                         699077 non-null  float64
 11  kitchen_living                  699077 non-null  float64
 12  living_bedroom

In [90]:
import math 

# We normalize x values to match with the 0-2π cycle
tiny_master_20["month_norm"] = 2 * math.pi * tiny_master_20["month"] / tiny_master_20["month"].max()

tiny_master_20["cos_month"] = np.cos(tiny_master_20["month_norm"])
tiny_master_20["sin_month"] = np.sin(tiny_master_20["month_norm"])

In [91]:
tiny_master_20["month_norm"]

12         4.188790
28         3.665191
32         1.570796
34         3.665191
44         3.141593
             ...   
1117196    6.283185
1117197    6.283185
1117198    5.235988
1117199    6.283185
1117200    6.283185
Name: month_norm, Length: 699077, dtype: float64

In [92]:
tiny_master_20['price_per_day'] = tiny_master_20['inquiry_price']/tiny_master_20['length_stay']

In [93]:
# generate the feature inquiry_price_per_day, max_price_per_day, min_price_per_day

medium_master_price_agg = tiny_master_20.groupby(['listing_id','month']).agg(['mean','max','min'])['price_per_day'].reset_index()

In [94]:
medium_master_price_agg

,listing_id,month,mean,max,min
0,00005de7-6ed8-541b-be83-2573a46cd307,5,96.041667,98.333333,93.750000
1,00005de7-6ed8-541b-be83-2573a46cd307,6,89.727891,98.333333,87.857143
2,00005de7-6ed8-541b-be83-2573a46cd307,7,103.958333,104.583333,103.333333
3,00005de7-6ed8-541b-be83-2573a46cd307,8,103.668367,107.857143,92.857143
4,00005de7-6ed8-541b-be83-2573a46cd307,10,87.857143,87.857143,87.857143
...,...,...,...,...,...
158850,ffffa8d1-d94a-5475-8e0d-2057495f4734,6,70.292208,78.571429,60.454545
158851,ffffa8d1-d94a-5475-8e0d-2057495f4734,7,80.408163,88.571429,35.714286
158852,ffffa8d1-d94a-5475-8e0d-2057495f4734,8,81.200000,88.571429,71.428571
158853,ffffa8d1-d94a-5475-8e0d-2057495f4734,9,64.047619,65.000000,63.571429


In [95]:
medium_master_inquiry_count = tiny_master_20.groupby(['listing_id','month']).agg(['count'])['price_per_day'].reset_index()

In [96]:
medium_master_inquiry_count

,listing_id,month,count
0,00005de7-6ed8-541b-be83-2573a46cd307,5,2
1,00005de7-6ed8-541b-be83-2573a46cd307,6,7
2,00005de7-6ed8-541b-be83-2573a46cd307,7,2
3,00005de7-6ed8-541b-be83-2573a46cd307,8,7
4,00005de7-6ed8-541b-be83-2573a46cd307,10,1
...,...,...,...
158850,ffffa8d1-d94a-5475-8e0d-2057495f4734,6,4
158851,ffffa8d1-d94a-5475-8e0d-2057495f4734,7,7
158852,ffffa8d1-d94a-5475-8e0d-2057495f4734,8,5
158853,ffffa8d1-d94a-5475-8e0d-2057495f4734,9,3


In [97]:
medium_master_1 = pd.merge(medium_master_price_agg, tiny_master_20, left_on=['listing_id','month'], right_on=['listing_id','month'], how='right')

In [98]:
medium_master_1.drop('Unnamed: 0',axis=1,inplace=True)

In [99]:
medium_master_2 = pd.merge(medium_master_inquiry_count, medium_master_1, left_on=[
    'listing_id','month'], right_on=['listing_id','month'], how='right')

In [100]:
medium_master_2.drop(['inquiry_price','month','month_norm','year','yearmonth'],axis=1,inplace=True)

In [101]:
medium_master_2.drop(['price_per_day'],axis=1,inplace=True)

In [102]:
medium_master_2.rename(columns={
    'count':'inquiry_count','mean':'mean_price_per_day','max':'max_price_per_day',
    'min':'min_price_per_day'},inplace=True)

In [103]:
list(medium_master_2.columns.values)

['listing_id',
 'inquiry_count',
 'mean_price_per_day',
 'max_price_per_day',
 'min_price_per_day',
 'result_views',
 'expose_views',
 '1_room_apartment',
 'dining_room',
 'en_suite_bathroom',
 'kitchen',
 'kitchen_living',
 'living_bedroom',
 'living_/_dining_room',
 'living_room',
 'roof-deck',
 'separate_WC',
 'veranda',
 'additional_bed',
 'bathtub',
 'bed_linen',
 'bicycles',
 'coffee_machine',
 'computer',
 'double_bed',
 'fan',
 'fireplace',
 'freezer',
 'games',
 'garage',
 'garden_furniture',
 'hair_dryer',
 'hot_tub',
 'infrared_cabin',
 'kingsize_bed',
 'microwave',
 'pool_table',
 'private_parking',
 'radio',
 'refrigerator',
 'shower',
 'single_bed',
 'sofa_bed',
 'steam_bath',
 'tanning_bed',
 'cooking',
 'date',
 'adult_count',
 'children_count',
 'pets_count',
 'arrival_date',
 'departure_date',
 'length_stay',
 'subregion',
 'property_type',
 'option_allergic',
 'option_non_smoking_only',
 'option_holiday_with_your_pet',
 'option_holiday_with_your_horse',
 'option_holi

In [104]:
medium_master_2.drop([
    'date','arrival_date','departure_date','option_holiday_with_your_dog',
    'option_holiday_with_your_horse'],axis=1,inplace=True)

In [105]:
# trying not to be a dummy by getting dummies

master_dummies = pd.get_dummies(medium_master_2, columns=['subregion','property_type'])

In [106]:
# save csv for this data set

master_dummies.to_csv('../data/master_with_dummies_2020.csv')

In [107]:
master_dummies

,listing_id,inquiry_count,mean_price_per_day,max_price_per_day,min_price_per_day,result_views,expose_views,1_room_apartment,dining_room,en_suite_bathroom,...,subregion_Westerzgebirge,subregion_Westheide,subregion_Willingen,subregion_Zeller Land,subregion_Zugspitz-Region,subregion_Überlinger See,property_type_apartment,property_type_bungalow,property_type_holiday_apartment,property_type_holiday_houses
0,97232bc1-cee6-54cc-9965-be13177051d3,4,133.607143,179.000000,0.000000,2985.0,630.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,97232bc1-cee6-54cc-9965-be13177051d3,9,143.579365,182.000000,0.000000,1862.0,257.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,97232bc1-cee6-54cc-9965-be13177051d3,10,73.507143,88.500000,0.000000,560.0,102.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,97232bc1-cee6-54cc-9965-be13177051d3,9,143.579365,182.000000,0.000000,1862.0,257.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,97232bc1-cee6-54cc-9965-be13177051d3,7,164.119048,179.000000,130.833333,2549.0,395.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699072,bd2737ac-54e2-5045-8706-bf0aef99456e,4,85.250000,94.000000,79.000000,344.0,83.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
699073,bd2737ac-54e2-5045-8706-bf0aef99456e,4,85.250000,94.000000,79.000000,344.0,83.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
699074,00779d9d-7264-5704-a71d-e6d1849cb4b7,1,133.333333,133.333333,133.333333,1239.0,180.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
699075,00779d9d-7264-5704-a71d-e6d1849cb4b7,1,140.000000,140.000000,140.000000,363.0,11.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [108]:
X = master_dummies.drop(['listing_id','inquiry_count'],axis=1)
Y = master_dummies['inquiry_count']

In [109]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [110]:
# we love to normalize our data!!

from sklearn.preprocessing import MinMaxScaler


In [111]:
the_list = [
    'result_views','expose_views','adult_count','children_count',
    'pets_count','length_stay','bathrooms','bedrooms','max_guests','living_area']

In [112]:
# Scaling with MinMaxScaler
scaler_norm = MinMaxScaler()
X_train_scaled_norm = scaler_norm.fit_transform(X_train[the_list])
X_test_scaled_norm = scaler_norm.transform(X_test[the_list])

In [113]:
# Concatenating normalized columns 
X_train_preprocessed_norm = np.concatenate([X_train_scaled_norm, X_train.drop(the_list, axis=1)], axis=1)
X_test_preprocessed_norm = np.concatenate([X_test_scaled_norm, X_test.drop(the_list, axis=1)], axis=1)

In [114]:
X_train_preprocessed_norm.shape

(489353, 270)

In [115]:
X_test_preprocessed_norm.shape

(209724, 270)

## Trainining the model and tracking with MLFlow

In [116]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run()
run = mlflow.active_run()

In [117]:
print("Active run_id: {}".format(run.info.run_id))

Active run_id: 3b2ed4087f834c89b4aeaa4ded80e311


In [118]:
master_dummies.isna().sum()

listing_id                         0
inquiry_count                      0
mean_price_per_day                 0
max_price_per_day                  0
min_price_per_day                  0
                                  ..
subregion_Überlinger See           0
property_type_apartment            0
property_type_bungalow             0
property_type_holiday_apartment    0
property_type_holiday_houses       0
Length: 272, dtype: int64

In [119]:
#training the model
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train_preprocessed_norm, y_train)

In [120]:
from sklearn.metrics import mean_squared_error
y_train_pred = reg.predict(X_train_preprocessed_norm)
rmse_train = mean_squared_error(y_train, y_train_pred,squared=False)
print(rmse_train)

52.595382382323926


In [121]:
y_test_pred = reg.predict(X_test_preprocessed_norm)
rmse_test = mean_squared_error(y_test, y_test_pred,squared=False)
print(rmse_test)

51.48302569286572


In [122]:
params = {
      "train_test_split": 30,
      "normalized data": 'yes',
      "2019 and 2020 data": "2019 and 2020 data", 
      "metric": 'rmse'
  }

In [123]:
mlflow.log_params(params)
mlflow.set_tag("running_from_jupyter", "baseline model 2020")
mlflow.log_metric("train -" + "RMSE", rmse_train)
mlflow.log_metric("test -" + "RMSE", rmse_test)
# mlflow.log_artifact("../models")
# mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [124]:
mlflow.get_run(run_id=run.info.run_id)

<Run: data=<RunData: metrics={'test -RMSE': 51.48302569286572, 'train -RMSE': 52.595382382323926}, params={'2019 and 2020 data': '2019 and 2020 data',
 'metric': 'rmse',
 'normalized data': 'yes',
 'train_test_split': '30'}, tags={'mlflow.source.git.commit': '9effae1b300b9decf3137cf7b3b71fae759dd20a',
 'mlflow.source.name': '/Users/lukeblecha/neuefische-traum/traum-preis/.venv/lib/python3.8/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'lukeblecha',
 'running_from_jupyter': 'baseline model 2020'}>, info=<RunInfo: artifact_uri='s3://neuefische-mlflow/mlflow-artifacts/25/3b2ed4087f834c89b4aeaa4ded80e311/artifacts', end_time=1626870966899, experiment_id='25', lifecycle_stage='active', run_id='3b2ed4087f834c89b4aeaa4ded80e311', run_uuid='3b2ed4087f834c89b4aeaa4ded80e311', start_time=1626870947664, status='FINISHED', user_id='lukeblecha'>>

## Checking the experiments

while the next cell is running you will not be able to run other cells in the notebook

In [125]:
#!mlflow ui